In [1]:
import pandas as pd
import janitor as jn
import numpy as np
from janitor.functions._numba import _numba_less_than, _numba_greater_than

In [2]:
# url = 'https://raw.githubusercontent.com/samukweku/data-wrangling-blog/master/notebooks/Data_files/results.csv'
# events = pd.read_csv(url, parse_dates=['start', 'end']).iloc[:, 1:]
# events.head()

In [3]:
# %%timeit
# (events
# .conditional_join(
#     events,
#     ('end', 'start', '>='),
#     ('start', 'end', '<='),
#     # ('id', 'id', '!='),
#     use_numba = False,
#     df_columns = ['id', 'start', 'end'],
#     right_columns = ['id', 'start', 'end'])
# )

In [4]:
# %%timeit
# (events
# .conditional_join(
#     events,
#     ('end', 'start', '>='),
#     ('start', 'end', '<='),
#     # ('id', 'id', '!='),
#     use_numba = True,
#     df_columns = ['id', 'start', 'end'],
#     right_columns = ['id', 'start', 'end'])
# )

In [5]:
# (events
# .conditional_join(
#     events,
#     ('end', 'start', '>='),
#     ('start', 'end', '<='),
#     # ('id', 'id', '!='),
#     use_numba = True,
#     df_columns = ['id', 'start', 'end'],
#     right_columns = ['id', 'start', 'end'])
# )

In [6]:
# (events
# .conditional_join(
#     events,
#     ('start', 'end', '<='),
#     ('end', 'start', '>='),
#     ('id', 'id', '!='),
#     use_numba = True,
#     df_columns = ['id', 'start', 'end'],
#     right_columns = ['id', 'start', 'end'])
# )

In [7]:
# %%timeit
# (events
# .conditional_join(
#     events,
#     ('start', 'end', '<='),
#     ('end', 'start', '>='),
#     ('id', 'id', '!='),
#     use_numba = True,
#     df_columns = ['id', 'start', 'end'],
#     right_columns = ['id', 'start', 'end'])
# )

In [8]:
# %%timeit
# (events
# .conditional_join(
#     events,
#     ('start', 'end', '<='),
#     ('end', 'start', '>='),
#     ('id', 'id', '!='),
#     use_numba = False,
#     df_columns = ['id', 'start', 'end'],
#     right_columns = ['id', 'start', 'end'])
# )

In [9]:

C=dict(key=[f"c{num}" for num in range(1,8)],
       vol=[35,15,5,35,18,90,17],
       profit=[45,35,55,12,15,55,11],
       unitsSold=[15,10,30,10,15,80,2]
       )
C=pd.DataFrame(C)


D=dict(key=[f'd{num}' for num in range(1,9)],
       vol=[20,50,15,16,40,20,40,2],
       profit=[30,10,12,52,35,20,30,57],
       unitsSold=[20,35,10,12,40,30,5,15]
       )
D=pd.DataFrame(D)

In [10]:
C

,key,vol,profit,unitsSold
0,c1,35,45,15
1,c2,15,35,10
2,c3,5,55,30
3,c4,35,12,10
4,c5,18,15,15
5,c6,90,55,80
6,c7,17,11,2


In [11]:
D

,key,vol,profit,unitsSold
0,d1,20,30,20
1,d2,50,10,35
2,d3,15,12,10
3,d4,16,52,12
4,d5,40,35,40
5,d6,20,20,30
6,d7,40,30,5
7,d8,2,57,15


In [12]:
# (array([0, 1, 2, 3, 4, 5]), array([0, 0, 0, 3, 0, 3, 0]))

In [13]:
C.sort_values('vol')

,key,vol,profit,unitsSold
2,c3,5,55,30
1,c2,15,35,10
6,c7,17,11,2
4,c5,18,15,15
0,c1,35,45,15
3,c4,35,12,10
5,c6,90,55,80


In [14]:
D.sort_values('vol')

,key,vol,profit,unitsSold
7,d8,2,57,15
2,d3,15,12,10
3,d4,16,52,12
0,d1,20,30,20
5,d6,20,20,30
4,d5,40,35,40
6,d7,40,30,5
1,d2,50,10,35


In [15]:
# C = pd.concat([C]*10_00)
# D = pd.concat([D]*1_0)

In [16]:
# rar=C.conditional_join(D, ('vol','vol','<'),('profit','profit','>'),use_numba=True)[0]
# rar[np.argsort(rar[:,1])]

In [17]:
# C.conditional_join(D, ('profit','profit','>'),('vol','vol','<'),use_numba=False,keep='first')

In [18]:
# C.conditional_join(D, ('profit','profit','>'),('vol','vol','<'),use_numba=True, keep='first')

In [19]:
C.conditional_join(D, ('vol','vol','<'),('profit','profit','>'), ('unitsSold','unitsSold','>'),use_numba=True)

l [[0 0 1]
 [1 2 3]
 [2 4 0]
 [2 3 2]
 [3 1 2]
 [3 4 3]
 [0 0 0]]
r [[0 0 1]
 [0 3 2]
 [1 0 2]
 [2 2 1]
 [2 2 0]
 [3 1 0]
 [3 2 3]
 [3 4 0]]
lb [ True  True False  True  True  True False]
rb [False  True  True  True  True  True  True  True]
lt [[0 0 1]
 [1 2 3]
 [2 3 2]
 [3 1 2]
 [3 4 3]]
rt [[0 3 2]
 [1 0 2]
 [2 2 1]
 [2 2 0]
 [3 1 0]
 [3 2 3]
 [3 4 0]]
li [0 1 3 4 5]
ri [1 2 3 4 5 6 7]


TypeError: tuple indices must be integers or slices, not str

In [ ]:
C.conditional_join(D, ('vol','vol','<'),('profit','profit','>'),use_numba=False, keep='first')

left                      right                     
   key vol profit unitsSold   key vol profit unitsSold
0   c1  35     45        15    d2  50     10        35
1   c2  15     35        10    d1  20     30        20
2   c3   5     55        30    d1  20     30        20
3   c4  35     12        10    d2  50     10        35
4   c5  18     15        15    d2  50     10        35
5   c7  17     11         2    d2  50     10        35

In [ ]:
# C.conditional_join(D, ('profit','profit','>'),('vol','vol','<'),use_numba=True, keep='last')

In [ ]:
# %timeit C.conditional_join(D, ('vol','vol','<'),('profit','profit','>'),use_numba=True, keep='all')

In [ ]:
df1 = pd.DataFrame({"id": [1, 1, 1, 2, 2, 3], "value_1": [2, 5, 7, 1, 23, 4]})
df2 = pd.DataFrame(
    {
        "id": [1, 1, 1, 1, 2, 2, 2, 3],
        "value_2A": [0, 3, 7, 12, 0, 2, 3, 1],
        "value_2B": [1, 5, 9, 15, 1, 4, 6, 3],
    }
)

In [ ]:
df1

,id,value_1
0,1,2
1,1,5
2,1,7
3,2,1
4,2,23
5,3,4


In [ ]:
df2

,id,value_2A,value_2B
0,1,0,1
1,1,3,5
2,1,7,9
3,1,12,15
4,2,0,1
5,2,2,4
6,2,3,6
7,3,1,3


In [ ]:
df1.sort_values('value_1')

,id,value_1
3,2,1
0,1,2
5,3,4
1,1,5
2,1,7
4,2,23


In [ ]:
df2.sort_values('value_2A')

,id,value_2A,value_2B
0,1,0,1
4,2,0,1
7,3,1,3
5,2,2,4
1,1,3,5
6,2,3,6
2,1,7,9
3,1,12,15


In [ ]:
df1.conditional_join(
    df2,
    ("value_1", "value_2B", "<"),
    ("value_1", "value_2A", ">"),
    use_numba=False,
    keep="first",
)

left         right                  
    id value_1    id value_2A value_2B
0    1       2     3        1        3
1    1       5     2        3        6
2    3       4     1        3        5

In [ ]:
df1.conditional_join(
    df2,
    ("value_1", "value_2B", "<"),
    ("value_1", "value_2A", ">"),
    use_numba=True,
    keep="all",
)

la [ 1  2  4  5  7 23]
ra [ 1  1  3  4  5  6  9 15]
la [ 1  2  4  5  7 23]
ra [ 0  0  1  2  3  3  7 12]


left         right                  
    id value_1    id value_2A value_2B
0    1       2     3        1        3
1    3       4     1        3        5
2    3       4     2        3        6
3    1       5     2        3        6

In [ ]:
# df1.conditional_join(
#     df2,
#     ("value_1", "value_2A", ">"),
#     ("value_1", "value_2B", "<"),
#     use_numba=True,
#     keep="last",
# )

In [ ]:
# df1.conditional_join(df2, ('value_1', 'value_2A','>'), ('value_1','value_2B', '<'), use_numba=False,keep='all')

In [ ]:
# df = {
#     "A": [1],
#     "B": [ 0.0
#           ],
#     "C": [""],
#     "D": [False],
# }
# df = pd.DataFrame(df)
# df

,A,B,C,D
0,1,0.0,,False


In [ ]:
# right = {
#     "Integers": [0],
#     "Numeric": [0.0],
#     # "Floats": [0.0,-1.0],
#     # "Strings": [""]*2,
#     # "Booleans": [False]*2,
#     # "Date": [pd.to_datetime("1970-01-01")]*2,
#     # "Dates_Right": [pd.to_datetime("1970-01-01")]*2,
# }
# right = pd.DataFrame(right)
# right

,Integers,Numeric
0,0,0.0


In [ ]:
# filters = ["A", "B", "Integers", "Numeric"]

# expected = df[["A", "B"]].merge(
#     right[["Integers", "Numeric"]], how="cross"
# )
# expected = expected.loc[
#         expected.A.ne(expected.Integers) & expected.B.ne(expected.Numeric)
#     ].sort_values(filters, ignore_index=True)

# expected

,A,B,Integers,Numeric


In [ ]:
# actual = (
#         df[["A", "B"]]
#         .conditional_join(
#             right[["Integers", "Numeric"]],
#             ("A", "Integers", "!="),
#             ("B", "Numeric", "!="),
#             how="inner",
#         )
#         # .sort_values(filters, ignore_index=True)
#     )


# actual

,A,B,Integers,Numeric


In [ ]:
# # df = df.sort_values("B").dropna(subset=["B"])
# actual = (
#         df[["A"]]
#         .conditional_join(
#             right[["Integers"]],
#             ("A", "Integers", "<"),
#             how="inner",
#             # keep="first",
#             use_numba=True,
#         )
#         # .sort_values(["A", "Integers"], ignore_index=True)
#     )

# actual

In [ ]:
# (
#         df[["A"]]
#         .conditional_join(
#             right[["Integers"]],
#             ("A", "Integers", "<"),
#             how="inner",
#             # keep="first",
#             use_numba=False,
#         )
#      #   .sort_values(["B", "Numeric"], ignore_index=True)
#     )

In [ ]:
# (
#         df[["A"]]
#         .merge(right[["Integers"]], how="cross")
#         .loc[lambda df: df.A.lt(df.Integers)]
#         .sort_values(["A", "Integers"], ignore_index=True)
#     )